<a href="https://colab.research.google.com/github/AlexandreES1996/CUDAteste/blob/main/petri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class lugar:
    def __init__(self, fichas=0):
        #Armazena o número de fichas atual
        #Nulo se não for especificado
        self.fichas = fichas

class trans:
	def __init__(self, arcosin=[], arcosout=[]):
        #Transições consistem em uma lista de arcos de entrada e uma de arcos de saída
        #Arcos são representados por tuplas (p, peso), onde p é o lugar de base

		self.arcosin = arcosin
		for i in range(len(self.arcosin)):
            #O usuário pode fornecer uma lista de lugares, ao invés de uma de tuplas
            #Neste caso, os lugares são substituídos por arcos de peso 1
			if not isinstance( (self.arcosin[i]), tuple):
				self.arcosin[i] = (self.arcosin[i], 1)
		self.arcosout = arcosout
		for i in range(len(self.arcosout)):
			if not isinstance( (self.arcosout[i]), tuple):
				self.arcosout[i] = (self.arcosout[i], 1)
    
    #Adiciona arco de entrada
	def addin(self, p, peso=1):
		self.arcosin.append( (p, peso) )
    #De saída
	def addout(self, p, peso=1):
		self.arcosout.append( (p, peso) )

    #Determina se a transição está atualmente habilitada.
    #Retornará True se, para todos os arcos de entrada, 
    #o número de fichas no lugar de base for maior ou igual ao peso
	def habilitada(self):
		h = bool(1)
		for (pin, peso) in self.arcosin:
			if peso > pin.fichas:
				h = bool(0)
		return h

    #Quando uma transição é disparada, fichas são removidas de todos os lugares
    #de entrada em quantidade igual aos pesos de seus arcos,
    #e fichas são adicionadas aos lugares de saída da mesma forma
	def dispara(self):
		if not self.habilitada():
			return
		else:
			for (pin, peso) in self.arcosin:
				pin.fichas -= peso
			for (pout, peso) in self.arcosout:
				pout.fichas += peso

In [ ]:
 #Esta classe descreve a estrutura de uma rede, em função de uma lista de lugares
#e de uma lista de transições entre estes.
#Uma instância pode ser definida passando lugares e transições como argumentos
#na declaração, ou por um método que a define em função de sua matriz de incidência
#Além disso, há um método que calcula e retorna esta matriz.
#Esta classe não descreve o estado ou funcionamento da rede.
#Para isto, utiliza-se a classe Simrede, que recebe uma rede como parâmetro

class Rede:
    #Redes consistem em uma lista de lugares e uma de transições conectadas.
    #Se a rede for definida recebendo estas duas listas diretamente,
    #todos os arcos de entrada e saída devem ser definidos previamente,
    #e todos os lugares conectados às transições devem ser passados.
    #Esta forma de definir é mais lenta, mas permite criar redes mais gerais,
    #e permite que lugares e transições tenham nomes únicos e possam ser
    #acompanhados individualmente.
    def __init__(self, parametrolistalugar=[],  parametrolistatrans=[]):
        self.listalugar = parametrolistalugar
        self.listatrans = parametrolistatrans
  

    #Além de passar os lugares e transições diretamente, pode-se definir uma
    #rede em termos de sua matriz de incidência com o método setmatriz.
    #Este método redefine a estrutura da rede, substituindo as
    #listas de lugares e transições se já existirem.
    #A matriz de entrada é representada como uma lista de listas de pesos.
    #Cada linha, ou elemento da lista principal, representa uma transição,
    #e cada coluna representa um lugar.
    #Matriz[i][j] representa o peso do arco entre a transição i e o lugar j,
    #sendo positivo se for um arco de saída, negativo se for de entrada,
    #e nulo se não houver arco.
    #Em uma rede criada desta forma, existe no máximo um arco entre uma transição
    #e um lugar qualquer.
    def setmatriz(self, matriz):
        self.matriz = matriz
        li = len(self.matriz)   #Número de linhas, e portanto de transições
        c = len(self.matriz[0]) #Número de colunas, e portanto de lugares

        #Cria-se um número de lugares igual ao de colunas
        self.listalugar = [None]*c
        for i in range(c):
            self.listalugar[i] = lugar(0)

        #Cria-se um número de transições igual ao de linhas
        self.listatrans = [None]*li
        for i in range(li):
            self.listatrans[i] = trans([],[])

        #Para cada elemento matriz[i][j], 
        #criar um arco entre a transição i e o lugar j.
        for i in range(li):
            for j in range(c):
                #O arco deve ser de entrada se o peso for negativo, e vice-versa
                if self.matriz[i][j] < 0:
                    self.listatrans[i].addin(self.listalugar[j], -self.matriz[i][j])
                elif self.matriz[i][j] > 0:
                    self.listatrans[i].addout(self.listalugar[j], self.matriz[i][j])


    #Retorna a matriz de incidência, calculada como função das listas de lugares
    #e transições.
    #Considerando os pesos de arcos de entrada como negativos, e os de saída
    #como positivos, o elemento [i][j] da matriz é a soma dos pesos dos arcos 
    #que ligam a i-ésima transição ao j-ésimo lugar
    def getmatriz(self):
        li = len(self.listatrans)  #O número de linhas é igual ao de transições
        self.matriz = [None]*li

        for i in range(li):
            #O número de colunas é igual ao de lugares
            #Para cada linha, cria-se uma lista vazia de tamanho igual
            self.matriz[i] = [0]*len(self.listalugar)

        for i in range(li):
            #Para cada arco de entrada da i-ésima transição,
            for(lugin, peso) in self.listatrans[i].arcosin:
                #obter o índice do lugar de origem
                j = self.listalugar.index(lugin)
                #subtrair o peso do arco do elemento correspondente da matriz
                self.matriz[i][j] -= peso

            #Igualmente para arcos de saída
            for(lugout, peso) in self.listatrans[i].arcosout:

                j = self.listalugar.index(lugout)

                self.matriz[i][j] += peso

        return self.matriz





In [ ]:
#Esta classe é utilizada para simular o funcionamento de uma rede.
#Ela é definida em função de uma instância de Rede, que descreve a estrutura da
#rede, e suporta métodos para ativar transições, acompanhar ou definir seu estado, 
#aplicar uma sequência de disparos qualquer, e mostrar seu grafo de 
#alcançabilidade a partir de um estado qualquer.

class Simrede:
    def __init__(self, rede):
        self.rede = rede    #rede a simular
        self.nlu = len(self.rede.listalugar)    #número de lugares
        self.ntr = len(self.rede.listatrans)    #número de transições
        self.listalugar = self.rede.listalugar 
        self.listatrans = self.rede.listatrans

        #estado, calculado a partir das fichas nos lugares da rede
        self.estado = [0]*self.nlu
        for i in range (self.nlu):
            self.estado[i] = self.listalugar[i].fichas

        #matriz de incidência
        self.matriz = self.rede.getmatriz()

        self.random = __import__('random')

    def passo(self):
        #simula o disparo aleatório de uma transição que esteja habilitada
        #se nenhuma estiver, nada acontece

        #lista de transições habilitadas
        self.transhabilitadas = []
        
        #buscam-se transições habilitadas, que são adicionadas à lista
        for i in range(self.ntr):
            if self.listatrans[i].habilitada():
                self.transhabilitadas.append(self.listatrans[i])

        #se houver transições habilitadas, dispara-se uma, escolhida
        #aleatóriamente
        if len(self.transhabilitadas) > 0:
            self.proxtrans = self.random.choice(self.transhabilitadas)
            self.proxtrans.dispara()

        #o estado é atualizado
        self.estado = self.getestado()

    #calcula e retorna o estado atual
    def getestado(self):
        for i in range (self.nlu):
            self.estado[i] = self.listalugar[i].fichas
        return self.estado

    #redefine o estado da rede para ser igual ao argumento recebido
    def setestado(self, estadonovo):
        self.estado = estadonovo
        for i in range (self.nlu):
            self.listalugar[i].fichas = self.estado[i]

    #recebe uma sequência de disparos, e dispara todas, 
    #mesmo se não estiverem habilitadas
    #o novo estado é o produto da sequência pela transposta da matriz de incidência
    def seqdisparo(self, seq):
        novoestado = self.getestado()
        for p in range(self.nlu):
            for t in range(self.ntr):
                novoestado[p] += seq[t]*self.matriz[t][p]

            self.setestado(novoestado)

    def _buscaestado(self, estado, grafo):
    #Dado um estado e um grafo de alcançabilidade, verifica se o dado estado 
    #já foi encontrado. Se houver já, retorna seu índice.
    #Senão, retorna o comprimento do grafo
        i = 0
        while i in range(len(grafo)):
            if grafo[i] == estado:
                return i
            i+=1
        return i


    def alcanca(self, origem=None):
    #Grafo de alcançabilidade
    #O estado recebido é a raiz, cada nó representa um estado, e os filhos de um
    #nó são os estados que podem resultar do disparo de uma transição a partir dele.
    #Este método imprime cada estado alcançável e os filhos deste estado.
    #Este método não termina se a rede for ilimitada
        if origem == None:
            self.origem = self.getestado()
        else:
            self.origem = origem.copy()
        self.estadobak = self.getestado().copy()
        self.setestado(self.origem)

        #Todo estado é armazenado uma vez em uma lista, self.grafo. Ela começa
        #pelo estado atual, ou por um estado arbitrário que pode ser passado como
        #parâmetro.
        self.grafo = [self.origem.copy()]

        #Para preencher esta lista, cada estado nela é examinado,
        #e seus filhos são adicionados ao fim da lista, se ainda não estiverem nela.
        #Depois não há mais nós a adicionar, o contador i chega ao fim da lista,
        #e o método termina.
        i = 0
        while i < len(self.grafo):
            #o i-ésimo estado em self.grafo será processado
            self.setestado(self.grafo[i].copy())

            #Imprime o estado sendo processado
            print("\nDe M[",i,"]: ",self.getestado(),":")
            #todas as transições da rede são examinadas
            for t in self.listatrans:
                #Se a transição t estiver habilitada, ela dispara, e o estado
                #resultante é impresso e adicionado ao grafo se ainda não estiver nela.
                if t.habilitada():
                    t.dispara()
                    self.estado = self.getestado()

                    it = self.listatrans.index(t) #índice da última transição

                    ind = self._buscaestado(self.estado, self.grafo)
                    if ind == len(self.grafo):
                        #Se o novo estado não estiver no grafo
                        self.grafo.append(self.estado)	#adiciona à lista

                    #Imprime o estado resultante
                    if self.estado == self.grafo[ind]:
                        print("\tt[",it,"] -> M[",ind,"]: ",self.estado)
                    #Se o estado resultante não estiver no grafo, mas todos os
                    #seus lugares possuírem um número de fichas maior ou igual
                    #ao de algum estado no grafo, imprime self.estado e o
                    #índice de seu equivalente.
                    else:
                        print("\tt[",it,"] -> M*[",ind,"]: ",self.estado)

                    #Volta para o i-ésimo estado, em que a rede estava
                    #antes do disparo:
                    self.setestado(self.grafo[i].copy())

            i += 1  #A evolução passa para o próximo estado em self.grafo

        #Imprime self.grafo, que contém todos os estados alcançáveis
        print("\nEstados alcançáveis: ", self.grafo)

        #Retorna ao estado em que a rede se encontrava antes do uso do método
        self.setestado(self.estadobak)

  

In [ ]:
#Rede usada na prova 1, definida em termos de transições e lugares

p1 = lugar(1)
p2 = lugar()
p3 = lugar()
p4 = lugar()
p5 = lugar()
p6 = lugar()
P = [p1, p2, p3, p4, p5, p6]

t1 = trans([p1], [p2, p3])
t2 = trans([p2], [p4])
t3 = trans([p3], [p5])
t4 = trans([p4, p5], [p6])
T = [t1, t2, t3, t4]

redeprova1A = Rede(P, T)

print("Matriz de incidência:\n")

S1A = Simrede(redeprova1A)

for i in range(S1A.ntr):
    print(S1A.matriz[i])

print("\nEstado:\n", S1A.getestado())

print("\nGrafo de alcançabilidade:")
S1A.alcanca()

sequencia = [1,1,1,1,1,1]
print("\n\nResultado da sequência de disparo", sequencia,":")
S1A.seqdisparo(sequencia)
print(S1A.estado)

print("\nFichas em p1:", p1.fichas)
print("Fichas em p6:", p6.fichas)


In [ ]:
#Rede usada na prova 1, definida em termos da matriz de incidência

matinc = [ [-1, 1, 1, 0, 0, 0], [0, -1, 0, 1, 0, 0], [0, 0, -1, 0, 1, 0], [0, 0, 0, -1, -1, 1] ]

redeprova2A = Rede()
redeprova2A.setmatriz(matinc)

print("Matriz de incidência:\n")

S2A = Simrede(redeprova2A)

for i in range(S2A.ntr):
    print(S2A.matriz[i])

S2A.setestado([1,0,0,0,0,0])
print("\nEstado:\n", S2A.getestado())

print("\nGrafo de alcançabilidade:")
S2A.alcanca()

novaorigem = [2,0,0,0,0,0]
print("\n\nGrafo de alcançabilidade para a partir do estado ",novaorigem,":")
S2A.alcanca(novaorigem)

sequencia = [1,1,1,1,1,1]
print("\n\nResultado da sequência de disparo", sequencia,":")
S2A.seqdisparo(sequencia)
print(S2A.estado)


In [ ]:
#Exemplo de rede com infinitos estados
#Há apenas dois lugares e duas transições. A primeira recebe uma ficha do 
#primeiro lugar e dá uma ao segundo, e a segunda transição recebe uma do
#segundo e dá duas ao primeiro lugar.

rede3 = Rede()
matinc = [ [-1,1], [2, -1] ]
rede3.setmatriz(matinc)

S3 = Simrede(rede3)

print("Matriz de incidência:\n")

for i in range(S3.ntr):
    print(S3.matriz[i])

print("\nEvolução a partir do estado [1, 0]")
S3.setestado([1,0])
print("M0: ",S3.estado)
S3.passo()
print("M1: ",S3.estado)
S3.passo()
print("M2: ",S3.estado)
S3.passo()
print("M3: ",S3.estado)
S3.passo()
print("M4: ",S3.estado)
S3.passo()
print("M5: ",S3.estado)
S3.passo()
print("M6: ",S3.estado)
print("A evolução nunca terminará\n")

S3.setestado([1,0])
print("Se ocorressem 20 transições de cada tipo, o estado resultante seria:")
S3.seqdisparo([20, 20])
print("M: ",S3.estado)